In [1]:
import tensorflow as tf
import os

2025-11-01 11:47:14.416537: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-01 11:47:14.458596: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-01 11:47:15.974856: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
# Convert .jpg files to .tfrec
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # BytesList requires bytes
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def image_to_tfexample(image_string, label):
    feature = {
        "image": _bytes_feature(image_string),
        "label": _bytes_feature(label.encode('utf-8'))
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

def write_tfrecord(image_dir, output_dir, label):
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"{label}.tfrec")

    with tf.io.TFRecordWriter(output_file) as writer:
        for filename in os.listdir(image_dir):
            if filename.lower().endswith(".jpg"):
                image_path = os.path.join(image_dir, filename)
                with open(image_path, "rb") as f:
                    image_bytes = f.read()
                tf_example = image_to_tfexample(image_bytes, label)
                writer.write(tf_example.SerializeToString())
    print(f"TFRecord saved to: {output_file}")

# Paths
photo_jpg_dir = "photo_jpg"
monet_jpg_dir = "monet_jpg"
photo_tfrec_dir = "photo_tfrec"
monet_tfrec_dir = "monet_tfrec"

# Convert
write_tfrecord(photo_jpg_dir, photo_tfrec_dir, "photo")
write_tfrecord(monet_jpg_dir, monet_tfrec_dir, "monet")
